In [2]:
from __future__ import absolute_import
import sys
import pandas as pd
import json
sys.path.append('/home/yichun/projects/information_retrieval')
from process import store_decomposed_sentences_df
from compute_similarity.compute import compute_similarities
from extractor_keywords.get_topic_based_on_keywords import get_extracted_keywords, get_topic
from evaluation import evaluate

CONFIG = json.load(open("/home/yichun/projects/information_retrieval/config.json"))

### 1) Remove frenquent sentences and store them in csv file

In [3]:
#store_decomposed_sentences_df('information_retrieval/data/judgements.jsonl') # uncomment this line the first time

In [4]:
judgements = pd.read_csv('/home/yichun/projects/information_retrieval/data/judgements')
judgements.head()

,judgementId,text
0,1,par mise à disposition au greffe déclare l'ass...
1,2,annule les fermes rappels des 4102010 23122010...
2,3,déclare recevable l'appel formé par la caisse ...
3,4,statuant par arrêt réputé contradictoire et en...
4,5,après en avoir délibéré conformément à la loi ...


### 2) Compute similarities between each judgement and 7 topics

In [5]:
judgements = compute_similarities(judgements, vectors=CONFIG['vectors_file'], method=CONFIG['sentence_similarity_method'])
judgements[['text', 'sim_a', 'sim_b', 'sim_c', 'sim_d', 'sim_e', 'sim_f', 'sim_g']].head()

,text,sim_a,sim_b,sim_c,sim_d,sim_e,sim_f,sim_g
0,par mise à disposition au greffe déclare l'ass...,0.247801,0.222064,-0.018451,0.071051,0.256975,0.222961,0.225813
1,annule les fermes rappels des 4102010 23122010...,0.457787,0.463368,0.168021,0.448591,0.730311,0.723619,0.740945
2,déclare recevable l'appel formé par la caisse ...,0.249044,0.248860,0.223531,0.215498,0.225614,0.120425,0.078748
3,statuant par arrêt réputé contradictoire et en...,0.112951,0.151913,-0.168179,-0.004682,0.159331,0.149859,0.180489
4,après en avoir délibéré conformément à la loi ...,0.300301,0.295410,0.003767,0.187835,0.492992,0.520812,0.542429


### 3) Extract keywords from judgements

In [8]:
judgements['keywords'] = judgements.apply(lambda row: get_extracted_keywords(row['text']), axis=1)
judgements[['text', 'keywords']].head(20)

,text,keywords
0,par mise à disposition au greffe déclare l'ass...,[]
1,annule les fermes rappels des 4102010 23122010...,"[travail, compensatrice, indemnité, préavis, l..."
2,déclare recevable l'appel formé par la caisse ...,[]
3,statuant par arrêt réputé contradictoire et en...,[]
4,après en avoir délibéré conformément à la loi ...,"[dommages et intérêts, à verser]"
5,statuant par arrêt réputé contradictoirement c...,"[travail, contrat, indemnité, licenciement, do..."
6,statuant dans les limites de l'appel infirme l...,[à payer]
7,déclare recevable l'appel formé par xxxx confi...,"[indemnité, à payer]"
8,statuant publiquement par mise à disposition a...,"[contrat, dommages et intérêts, congés payés, ..."
9,statuant sur les seuls éléments déférés par la...,"[dommages et intérêts, congés payés, à payer]"


### 4) Identify topic by keywords

In [9]:
judgements['topic_keywords'] = judgements.apply(lambda row: get_topic(row['keywords']), axis=1)
judgements[['text', 'keywords', 'topic_keywords']].head(20)

,text,keywords,topic_keywords
0,par mise à disposition au greffe déclare l'ass...,[],[]
1,annule les fermes rappels des 4102010 23122010...,"[travail, compensatrice, indemnité, préavis, l...","[A, B, D, E, F, G]"
2,déclare recevable l'appel formé par la caisse ...,[],[]
3,statuant par arrêt réputé contradictoire et en...,[],[]
4,après en avoir délibéré conformément à la loi ...,"[dommages et intérêts, à verser]",[]
5,statuant par arrêt réputé contradictoirement c...,"[travail, contrat, indemnité, licenciement, do...","[A, B, D, E]"
6,statuant dans les limites de l'appel infirme l...,[à payer],[]
7,déclare recevable l'appel formé par xxxx confi...,"[indemnité, à payer]",[]
8,statuant publiquement par mise à disposition a...,"[contrat, dommages et intérêts, congés payés, ...",[F]
9,statuant sur les seuls éléments déférés par la...,"[dommages et intérêts, congés payés, à payer]",[F]


### 5 Sort result by difference between top1 topic similarity and top2 topic similarity 

In [10]:
judgements = judgements.sort_values(by='diff_top1sim_top2sim', ascending=True)
judgements[['text', 'sim_a', 'sim_b', 'sim_c', 'sim_d', 'sim_e', 'sim_f', 'sim_g','diff_top1sim_top2sim']].head()

,text,sim_a,sim_b,sim_c,sim_d,sim_e,sim_f,sim_g,diff_top1sim_top2sim
413,confirme le jugement entrepris sauf en ce qu'i...,0.237748,0.266127,-0.076346,0.213882,0.486552,0.519894,0.519892,0.000001
1228,ordonnons la prolongation du maintien de x se ...,0.103795,0.116931,0.065439,0.116933,0.114764,0.013696,0.036533,0.000002
688,donne acte à xxxx de son désistement d'appel d...,0.170396,0.170390,0.078208,0.026875,0.051081,-0.171795,-0.178366,0.000006
9841,statuant contradictoirement et après débats en...,0.181684,0.182589,0.117369,0.106396,0.182595,0.139111,0.094079,0.000006
5040,infirme le jugement entrepris en ce qu'il a co...,0.313827,0.344019,0.051794,0.298217,0.578163,0.578176,0.576485,0.000013


### 6 Store it in excel file

In [11]:
writer = pd.ExcelWriter("/home/yichun/projects/information_retrieval/result.xlsx", engine='xlsxwriter')
judgements[['judgementId', 'text', 'keywords', 'topic_keywords', 'topic_similarity','sim_a','sim_b', 'sim_c', 'sim_d',
            'sim_e', 'sim_f', 'sim_g', 'diff_top1sim_top2sim']].to_excel(writer,sheet_name = CONFIG['sentence_similarity_method'])
writer.save()

### 7) Give new sentences and show : sentence, topic related, (dis)similarities computed by cosine similarity and wmd method, keywords, topic identified by similarity and by keywords

In [12]:
evaluate()



Sentence = ALORS QU’au titre de l’abus de position dominante tiré de ce que la société Google avait procédé à des coupures intempestives, sans motif et à répétition, la société E-Kanopi faisait valoir dans ses écritures d’appel que les coupures des 13 et 14 mai 2010 étaient abusives comme étant intervenues sans préavis et sans justification objective ;
Topic = D
Cosine Similarities = [0.4781866669654846, 0.43202903866767883, 0.43602117896080017, 0.3890143036842346, 0.4473118484020233, 0.41880741715431213, 0.38565436005592346]
Topic identified by similarities = A

Word Mover's Distances = [18.729119879049335, 18.64519992260676, 17.396262414855705, 18.019499687079044, 17.680443856291884, 17.29256555489971, 16.501698567275458]
Topic identified by dissimilarities = G

Keywords = ['préavis', 'abusive']
Topic indentified by keywords= []


Sentence = et débouter la salariée de ses demandes de rappels de salaire et congés payés afférents, de prime annuelle (treizième mois) et congés payés af

Word Mover's Distances = [18.1282284453897, 17.98565764704046, 16.814023733187476, 16.88744320955054, 16.00853093036578, 12.336193239554381, 14.453995519354734]
Topic identified by dissimilarities = F

Keywords = ['dommages-intérêts', 'congés payés']
Topic indentified by keywords= ['F']




Sentence = par ces motifs la cour confirme le jugement déféré sauf en ses dispositions relatives à la date des effets du divorce et à la prestation compensatoire infirmant sur ces chefs et statuant à nouveau fixe la date des effets du divorce au 31 décembre 2010 condamne xxxx à payer à yxxx la somme de 180000 € à titre de prestation compensatoire dit n'y avoir lieu à application des dispositions de l'article 700 du code de procédure civile laisse à chacun la charge de ses dépens d'appel
Topic = E
Cosine Similarities = [0.45167461037635803, 0.42180490493774414, 0.35146060585975647, 0.3724234700202942, 0.5252044200897217, 0.40782472491264343, 0.41461220383644104]
Topic identified by similarities = E

Word Mover's Distances = [20.184919834136963, 20.217316360473628, 18.923238945007324, 19.406355135722663, 18.617254429556457, 18.50054646492004, 18.070828390062502]
Topic identified by dissimilarities = G

Keywords = ['prestation', 'divorce', 'à payer']
Topic indentified by keywords= ['E']



Sentence = La somme de 22.539,99 euros au titre de l’indemnité de loyers dus jusqu’à la date d’échéance du bail suite à la rupture du bail ; ® La somme de 1.594,81 euros au titre de la perte de budget vacances 2012 suite à la rupture du bail
Topic = E
Cosine Similarities = [0.3243754208087921, 0.37881070375442505, 0.20906350016593933, 0.4180598258972168, 0.5549786686897278, 0.5065261125564575, 0.48155519366264343]
Topic identified by similarities = E

Word Mover's Distances = [19.72804489643009, 18.937426396012654, 18.253685857781097, 17.624990419556944, 16.758157869843085, 17.82612927896543, 17.527275978496156]
Topic identified by dissimilarities = E

Keywords = ['rupture', 'indemnité', 'bail']
Topic indentified by keywords= ['D', 'E']
